<a href="https://colab.research.google.com/github/fasghq/TT/blob/master/TT_Lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flax

     |████████████████████████████████| 92kB 6.3MB/s 


In [3]:
from jax import ops
import jax.numpy as jnp
from jax.numpy import maximum
from jax import grad
import jax.numpy as np
from jax.numpy import linalg as la
from jax import jit
from jax import lax
import numpy as nnp
import math
import time
import flax
from flax import struct

In [4]:
class TT_Tensor:
    def __init__(self, core, d, n, r):
        self.core = core
        self.d = d
        self.n = n
        self.r = r

In [5]:
@struct.dataclass
class Model():
    core: list

In [6]:
def mod_to_old(ttm):
    d = len(ttm.core)
    n = []
    r = []
    for X in ttm.core:
        n.append(X.shape[1])
        r.append(X.shape[0])
    r.append(ttm.core[d - 1].shape[2])
    return TT_Tensor(ttm.core, d, n, r)

In [7]:
def tt_get_d(tt):
  return len(tt.core)

In [8]:
def tt_get_n(tt):
  d = len(tt.core)
  n = []
  for X in tt.core:
        n.append(X.shape[1])
  return n

In [9]:
def tt_get_r(tt):
  d = len(tt.core)
  r = []
  for X in tt.core:
    r.append(X.shape[0])
  r.append(tt.core[d - 1].shape[2])
  return r

In [10]:
def tt_to_tensor(tt, shape):
    G = tt.core
    B = G[0]
    for i in range(1, len(G)):
        X = np.reshape(G[i], (G[i].shape[0], G[i].shape[1] * G[i].shape[2]))
        B = np.reshape(B, (int(np.size(B) / G[i].shape[0]), G[i].shape[0]))
        B = np.dot(B, X)
    B = np.reshape(B, shape)
    return B

In [11]:
def tt_svd(A, delta):

    #time1 = time.perf_counter()

    d = len(A.shape)
    N = np.size(A)
    n = A.shape

    C = A
    G = [] 
    r = [] 
    rk1 = 1

    for k in range(1, d):
        C = np.reshape(C, (rk1 * n[k-1], int(N / (rk1 * n[k-1]))))      
        u, s, v = la.svd(C, full_matrices=False)
        rc = s.shape[0]  - (s < delta).sum()
        r.append(rc)
        G.append(np.reshape(u, (rk1, n[k-1], u.shape[1])))
        s = np.diag(s)
        C = np.dot(s, v)
        rk = C.shape[0]
        N = (N * rk) / (n[k-1] * rk1)
        rk1 = rk
    
    if len(C.shape) == 2:
        C = np.reshape(C, (C.shape[0], C.shape[1], 1))
        r.append(C.shape[2])
        
    G.append(C)

    #time2 = time.perf_counter()
    #print("\n time: ", time2 - time1)
    
    return Model(G), r

In [12]:
def tt_sum(tt1, tt2):
    #time1 = time.perf_counter()
    d1 = len(tt1.core)
    d2 = len(tt2.core)
    if d1 == d2:
        d = d1
    else:
        raise Exception("Different dimensions of tensors.")

    r1 = []
    n1 = []
    for X in tt1.core:
        n1.append(X.shape[1])
        r1.append(X.shape[0])
    r1.append(tt1.core[len(tt1.core) - 1].shape[2])

    r2 = []
    n2 = []
    for X in tt2.core:
        n2.append(X.shape[1])
        r2.append(X.shape[0])
    r2.append(tt2.core[len(tt2.core) - 1].shape[2])
    
    r = [x + y for x, y in zip(r1, r2)]
    
    if r1[0] == r2[0]:
        r[0] = r1[0]
    else:
        raise Exception("Different sizes of first mode.")
        
    if r1[d] == r2[d]:
        r[d] = r1[d]
    else:
        raise Exception("Different sizes of last mode.")
        
    n = n1
    G = []
    
    for i in range(0, d):
        Gi = nnp.zeros((r[i], n[i], r[i+1]))
        Gi = ops.index_update(Gi, ops.index[0 : r1[i], :, 0 : r1[i+1]], tt1.core[i])
        Gi = ops.index_update(Gi, ops.index[r[i] - r2[i] : r[i], :, r[i+1] - r2[i+1]: r[i+1]], tt2.core[i])
        #Gi[0 : r1[i], :, 0 : r1[i+1]] = tt1.core[i]
        #Gi[r[i] - r2[i] : r[i], :, r[i+1] - r2[i+1]: r[i+1]] = tt2.core[i]   
        G.append(Gi)

    #time2 = time.perf_counter()
    #print("\n time: ", time2 - time1)
    
    return Model(G)

In [13]:
def tt_round(tt, delta):  
    G = tt.core
    d = len(tt.core)
    rn = []
    #time1 = time.perf_counter()    
    
    for k in range(d-1, 0, -1):
        r1, n, r2 = G[k].shape
        G[k] = np.reshape(G[k], (r1, n * r2))
        G[k] = np.transpose(G[k])
        G[k], R = la.qr(G[k])
        G[k] = np.transpose(G[k])
        G[k-1] = np.einsum('ijk,lk->ijl', G[k-1], R) 
    
    for k in range(0, d - 1):
        
        if k == 0:
            r1, n1, r2 = G[k].shape
            r3 = G[k+2].shape[0]
            n2 = int(G[k+1].shape[1] / r3)
        elif k == d - 2:
            r1 = G[k].shape[0]
            r2 = G[k+1].shape[0]
            n1 = int(G[k].shape[1] / r2)
            r3 = 1
            n2 = G[k+1].shape[1]
        else:
            r1 = G[k].shape[0]
            r2 = G[k+1].shape[0]
            n1 = int(G[k].shape[1] / r2)
            r3 = G[k+2].shape[0]
            n2 = int(G[k+1].shape[1] / r3)

        G[k] = np.reshape(G[k], (r1 * n1, r2))

        u, s, v = la.svd(G[k], full_matrices=False)
 
        rc = s.shape[0]  - (s < delta).sum()
        rn.append(rc)
        s = np.diag(s)  
         
        v = np.dot(s, v) 
        G[k] = np.reshape(u, (r1, n1, u.shape[1]))
        G[k+1] = np.einsum('ij,ik->kj', G[k+1], v.T)
    
    
    if len(G[d-1].shape) == 2:
        G[d-1] = np.reshape(G[d-1], (G[d-1].shape[0], G[d-1].shape[1], 1))

    #time2 = time.perf_counter()
    #print("\n time: ", time2 - time1)
    
    return Model(G), rn

In [45]:
def tt_matrix(A):
  d = int(len(A.shape) / 2)
  nper = []
  for i in range(d):
    nper.append(i)
    nper.append(i+d)
  return A.transpose(nper)

In [46]:
'''
A = []
dim = 2
x = 0
for i in range(2 ** dim):
  A.append(math.sin(x))
  x = x + math.pi / (2 ** dim - 1)
shp = [2 for i in range(dim)]
A = np.asarray(A)
A = A.reshape(shp)
'''
#A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
dim = 4
A = np.arange(2**(dim))
A = np.asarray(A)
A = A.reshape([2 for i in range(dim)])
eps = 1e-2 

print(A, "\n", A.shape)

A = tt_matrix(A)

print(A, "\n", A.shape)

fast_tt_svd = jit(tt_svd, static_argnums=(1))
tt, rn = fast_tt_svd(A, eps / math.sqrt(len(A.shape) - 1))
#ttnew, rn = tt_svd(A, eps / math.sqrt(len(A.shape) - 1))

for i in range(len(rn)-1):  
    tt.core[i] = tt.core[i][:, :, :rn[i]]
    tt.core[i+1] = tt.core[i+1][:rn[i], :, :]

for X in tt.core:
  print("c:\n", X)

print(tt_get_r(tt))

B = tt_to_tensor(tt, A.shape)
print(B)



[[[[ 0  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]]


 [[[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]] 
 (2, 2, 2, 2)
[[[[ 0  1]
   [ 4  5]]

  [[ 2  3]
   [ 6  7]]]


 [[[ 8  9]
   [12 13]]

  [[10 11]
   [14 15]]]] 
 (2, 2, 2, 2)
c:
 [[[ 0.31874645  0.94784003]
  [ 0.94784003 -0.31874645]]]
c:
 [[[ 0.63188636 -0.157323  ]
  [ 0.7734447   0.11531131]]

 [[-0.03911228 -0.7579181 ]
  [ 0.03119771 -0.6225059 ]]]
c:
 [[[ 0.54600716 -0.30995786]
  [ 0.8309377   0.15788099]]

 [[ 0.08937807  0.77318436]
  [-0.05856603  0.53026927]]]
c:
 [[[23.651743  ]
  [26.067415  ]]

 [[ 0.7713588 ]
  [-0.69987684]]]
[1, 2, 2, 2, 1]
[[[[1.4305115e-06 1.0000032e+00]
   [4.0000062e+00 5.0000067e+00]]

  [[2.0000043e+00 3.0000057e+00]
   [6.0000057e+00 7.0000057e+00]]]


 [[[7.9999952e+00 8.9999971e+00]
   [1.2000001e+01 1.3000000e+01]]

  [[9.9999981e+00 1.0999998e+01]
   [1.3999994e+01 1.4999993e+01]]]]


In [39]:
x = np.zeros((1, 2, 3, 4, 5, 6, 7, 8, 9, 10))
print(x.shape, " old")
d = int(len(x.shape) / 2)
b = []
for i in range(d):
  b.append(i)
  b.append(i+d)
print(b)
x = x.transpose(b)

print(x.shape, " new")

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  old
[0, 5, 1, 6, 2, 7, 3, 8, 4, 9]
(1, 6, 2, 7, 3, 8, 4, 9, 5, 10)  new


In [ ]:
# input tensor A
'''
A = np.array([[1/2, 1/3, 1/4], [1/3, 1/4, 1/5], [1/4, 1/5, 1/6]])
A = np.arange(24)
B = A.reshape(4, 3, 2)
A = B
'''

'''
A = np.ones(1000000)
A = A.reshape(100, 100, 100)
#4 3 2
'''
'''
A = []
for i in range(100):
  for j in range(100):
    for q in range(100):
      A.append(1 / (i + j + q + 3))
A = np.asarray(A)
A = A.reshape(100, 100, 100)
'''

A = []
dim = 15
x = 0
for i in range(2 ** dim):
  A.append(math.sin(x))
  x = x + math.pi / (2 ** dim - 1)
shp = [2 for i in range(dim)]
A = np.asarray(A)
A = A.reshape(shp)

eps = 1e-2 # accuracy


fast_tt_svd = jit(tt_svd, static_argnums=(1))
tt, rn = fast_tt_svd(A, eps / math.sqrt(len(A.shape) - 1))
#ttnew, rn = tt_svd(A, eps / math.sqrt(len(A.shape) - 1))

for i in range(len(rn)-1):  
    tt.core[i] = tt.core[i][:, :, :rn[i]]
    tt.core[i+1] = tt.core[i+1][:rn[i], :, :]

fast_tt_sum = jit(tt_sum)
%timeit tts = fast_tt_sum(tt, tt)
#%timeit tts = tt_sum(tt, tt)
#for X in tts.core:
#  print(X)

ttsold = mod_to_old(tts)
print("\nRanks tts: ", ttsold.r)

fast_tt_round = jit(tt_round, static_argnums=(1))
ttr, rn = fast_tt_round(tts, eps / math.sqrt(len(tts.core) - 1))
#ttr, rn = tt_round(tts, eps / math.sqrt(len(tts.core) - 1))

for i in range(len(rn)):  
    ttr.core[i] = ttr.core[i][:, :, :rn[i]]
    ttr.core[i+1] = ttr.core[i+1][:rn[i], :, :]


#ttr2 = ttpy.matrix.round(tts)
#ttr2 = ttpy.tensor.round(tts, eps)
'''
print("\nttr:")
for X in ttr.core:
    print("\nCore:\n", X, "\nShape: ", X.shape)
'''
ttro = mod_to_old(ttr)
print("\nRanks ttr: ", ttro.r)
'''
B = tt_to_tensor(tts, A.shape)
#print("\nB = A + A:\n", B)
C = tt_to_tensor(ttro, A.shape)
#print("\nC = round(B):\n", C)
print("\n", la.norm(B-C), " <= ", eps*la.norm(B), la.norm(B-C) <= eps*la.norm(B))
'''


The slowest run took 199.50 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 870 µs per loop

Ranks tts:  [1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1]

Ranks ttr:  [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


'\nB = tt_to_tensor(tts, A.shape)\n#print("\nB = A + A:\n", B)\nC = tt_to_tensor(ttro, A.shape)\n#print("\nC = round(B):\n", C)\nprint("\n", la.norm(B-C), " <= ", eps*la.norm(B), la.norm(B-C) <= eps*la.norm(B))\n'